In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [6]:
train_data_dir = '/content/dataset valid/train_224'
val_data_dir = '/content/dataset valid/val'
img_width, img_height = 224, 224
batch_size = 32

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(

    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,

    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,

    class_mode='categorical'
)

Found 2586 images belonging to 3 classes.
Found 53 images belonging to 3 classes.


In [10]:
# Pilih model pre-trained, misal MobileNetV2
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(img_width, img_height, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze layer pada base model
base_model.trainable = False

# Tambahkan layer baru di atas base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)  # Sesuaikan dengan jumlah kelas

model = Model(inputs=base_model.input, outputs=predictions)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 10  # Sesuaikan jumlah epoch

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples
 // batch_size
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


80/80 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.7529 - loss: 0.5972 - val_accuracy: 1.0000 - val_loss: 0.0814
Epoch 2/10
 1/80 ━━━━━━━━━━━━━━━━━━━━ 3:15 2s/step - accuracy: 0.9688 - loss: 0.1677

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - accuracy: 0.9688 - loss: 0.1677 - val_accuracy: 0.9048 - val_loss: 0.3325
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.9742 - loss: 0.1000
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 1.0000 - loss: 0.0272 - val_accuracy: 0.9688 - val_loss: 0.0932
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 124s 2s/step - accuracy: 0.9886 - loss: 0.0482 - val_accuracy: 1.0000 - val_loss: 0.0183
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 665us/step - accuracy: 0.9688 - loss: 0.1241
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.9975 - loss: 0.0228 - val_accuracy: 0.9688 - val_loss: 0.0416
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 1.0000 - loss: 0.0314 - val_accuracy: 1.0000 - val_loss: 0.0325
Epoch 9/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 138s 2s/step - accuracy: 0.9986 - loss: 0.0137
Epoch 10/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 1.0000 - loss: 0.0133 - val_accuracy: 0.9375 - val_loss:

In [12]:
model.save('INJUREAL.h5')

In [20]:
from tensorflow.keras.preprocessing import image
from google.colab import files
model = tf.keras.models.load_model('INJUREAL.h5')

# Upload gambar
uploaded = files.upload()

# Preprocessing gambar
for fn in uploaded.keys():
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)

  print(classes)

  # Tampilkan hasil prediksi
  class_labels = ['Abrasions', 'Bruises', 'Burns']
  predicted_class = class_labels[np.argmax(classes)]
  print(f"Gambar {fn} terdeteksi sebagai: {predicted_class}")

Saving abrasions (3)_horizontal_flip.jpg to abrasions (3)_horizontal_flip.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
abrasions (3)_horizontal_flip.jpg
[[0.02114417 0.7584419  0.22041392]]
Gambar abrasions (3)_horizontal_flip.jpg terdeteksi sebagai: Bruises
